<a href="https://colab.research.google.com/github/IzaakGagnon/Integrated_Information_Testing/blob/main/Partition_Methods/All_Directional_Bipartition_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!python -m pip install -U git+https://github.com/wmayner/pyphi.git@feature/iit-4.0
import numpy as np
import pyphi
import random
import time
pyphi.config.PROGRESS_BARS = False
pyphi.config.PARALLEL = False
pyphi.config.SHORTCIRCUIT_SIA = False
pyphi.config.WELCOME_OFF = True
pyphi.config.REPERTOIRE_DISTANCE = "GENERALIZED_INTRINSIC_DIFFERENCE"

  Cloning https://github.com/wmayner/pyphi.git (to revision feature/iit-4.0) to /tmp/pip-req-build-jd3iqqln
  Running command git clone --filter=blob:none --quiet https://github.com/wmayner/pyphi.git /tmp/pip-req-build-jd3iqqln
  Resolved https://github.com/wmayner/pyphi.git to commit 6b83cbdbbcdca75289415fe096adbac5f2ec7a4d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done

Welcome to PyPhi!

If you use PyPhi in your research, please cite the paper:

  Mayner WGP, Marshall W, Albantakis L, Findlay G, Marchman R, Tononi G.
  (2018). PyPhi: A toolbox for integrated information theory.
  PLOS Computational Biology 14(7): e1006343.
  https://doi.org/10.1371/journal.pcbi.1006343

Documentation is available online (or with the built-in `help()` function):
  https://pyphi.readthedocs.io

To report issues, please use the issue tracker on the GitHub repository:
  https://github.com/wmayner/pyphi

For general d

In [2]:
import functools
import itertools
from itertools import chain, product
import numpy as np
from more_itertools import distinct_permutations
from toolz import unique
from pyphi import combinatorics
from pyphi.cache import cache
from pyphi.conf import config, fallback
from pyphi.direction import Direction
from pyphi.models.cuts import (
    Bipartition,
    CompleteGeneralKCut,
    CompleteGeneralSetPartition,
    Cut,
    GeneralKCut,
    GeneralSetPartition,
    KPartition,
    Part,
    SystemPartition,
    Tripartition,
)
from pyphi.partition import system_partition_types
from pyphi.registry import Registry

In [3]:
def normalize_rows(matrix):
    ### Scales probabilities in a matrix so that they satisfy the markov property. Proof that CI still holds not completed yet
    num_rows = matrix.shape[0]
    normalized_matrix = np.zeros_like(matrix)  # Create an empty matrix of the same shape
    for i in range(num_rows):
        row = matrix[i, :]
        current_sum = np.sum(row)
        if current_sum > 0:
            scaling_factor = 1.0 / current_sum
            normalized_row = row * scaling_factor
            normalized_matrix[i, :] = normalized_row
        else: print("Zero_Sum_Error: Problem With Values Generated")
    return normalized_matrix
def create_noisy_network(size):
    ### Obtains a random network of nodes of a given size, directly ready for computing Phi.
    return pyphi.Network(normalize_rows(pyphi.convert.state_by_node2state_by_state(np.random.rand(2**size,size))))
def create_noiseless_network(size):
  return pyphi.Network(np.random.randint(0,2,(2**size,size)))

In [4]:
from itertools import combinations
def bipartitions(collection):
    collection = list(collection)
    n = len(collection)

    # Special case: no bipartitions possible if the collection has fewer than 2 elements
    if n < 2:
        return

    # Generate all combinations for the first subset of size from 1 to n-1
    for i in range(1, n):
        for first_subset in combinations(collection, i):
            second_subset = [item for item in collection if item not in first_subset]
            yield [list(first_subset), second_subset]

def _unidirectional_set_bipartitions(node_indices, node_labels=None):
    """Generate all unidirectional set partitions of a set of nodes."""
    if len(node_indices) == 1 or config.SYSTEM_PARTITION_INCLUDE_COMPLETE:
        yield CompleteGeneralSetPartition(node_indices, node_labels=node_labels)
    _node_indices = set(range(len(node_indices)))
    for partition in bipartitions(_node_indices):
        for directions in product(Direction.all(), repeat=len(partition)):
            cut_matrix = np.zeros([len(_node_indices), len(_node_indices)], dtype=int)
            for part, direction in zip(partition, directions):
                nonpart = list(_node_indices - set(part))
                if direction == Direction.CAUSE:
                    source, target = nonpart, part
                else:
                    source, target = part, nonpart
                cut_matrix[np.ix_(source, target)] = 1
                if direction == Direction.BIDIRECTIONAL:
                    cut_matrix[np.ix_(target, source)] = 1
            yield GeneralSetPartition(
                node_indices,
                cut_matrix,
                node_labels=node_labels,
                set_partition=partition,
            )

@system_partition_types.register("ALL_BIPARTITIONS")
@functools.wraps(_unidirectional_set_bipartitions)
def unidirectional_set_partitions(node_indices, node_labels=None):
    # TODO(4.0) generate properly without using set
    yield from unique(
        _unidirectional_set_bipartitions(node_indices, node_labels=node_labels)
    )




In [7]:
np.random.seed(4212297)
size = 4
network = create_noisy_network(size)
state = [np.random.choice([0, 1]) for i in range(size)]
subsystem_cause = pyphi.Subsystem(network,state,backward_tpm=True)
subsystem_effect = pyphi.Subsystem(network,state,backward_tpm=False)

pyphi.config.SYSTEM_PARTITION_TYPE = "SET_UNI/BI"
s = time.time()
sia = pyphi.backwards.sia(subsystem_cause,subsystem_effect)
f = time.time()
print(sia)
print(pyphi.config.SYSTEM_PARTITION_TYPE,f - s)

pyphi.config.SYSTEM_PARTITION_TYPE = "ALL_BIPARTITIONS"
s = time.time()
sia = pyphi.backwards.sia(subsystem_cause,subsystem_effect)
f = time.time()
print(sia)
print(pyphi.config.SYSTEM_PARTITION_TYPE,f - s)

┌───────────────────────────────────────┐
│     SystemIrreducibilityAnalysis      │
│ ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  │
│      Subsystem:  n0,n1,n2,n3          │
│  Current state:  (0,1,0,0)            │
│            φ_s: 0.24366530355074012   │
│ Normalized φ_s: 0.030458162943842515  │
│          CAUSE:  (1,1,0,1)            │
│           II_c: 2.125095224353553     │
│         EFFECT:  (0,0,1,1)            │
│           II_e: 0.5349762313772047    │
│   #(tied MIPs): 0                     │
│      Partition:                       │
│                 3 parts: {n0,n1n2,n3} │
│                 [[0 0 0 1]            │
│                  [1 0 0 1]            │
│                  [1 0 0 1]            │
│                  [1 1 1 0]]           │
└───────────────────────────────────────┘
SET_UNI/BI 2.2780721187591553
┌──────────────────────────────────────┐
│     SystemIrreducibilityAnalysis     │
│ ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ │
│      Subsystem:  n0,n1,n2,n3         │
│  Curre